<a href="https://colab.research.google.com/github/Fahad8389/RFantibody-Colab/blob/main/RFantibody_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Colab Setup

**GPU Required:** Before running, enable GPU runtime:
1. Go to **Runtime → Change runtime type**
2. Select **T4 GPU** (or better)
3. Click **Save**

In [1]:
#@title 1. Clone RFantibody Repository
#@markdown This will clone the RFantibody repository from GitHub.

#@title 1. Clone RFantibody Repository
import os

# Clone the repository
!git clone https://github.com/RosettaCommons/RFantibody.git
os.chdir("/content/RFantibody")

print("✓ Repository cloned")
print("Current directory:", os.getcwd())

Cloning into 'RFantibody'...
remote: Enumerating objects: 957, done.
remote: Counting objects: 100% (256/256), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 957 (delta 152), reused 129 (delta 115), pack-reused 701 (from 1)
Receiving objects: 100% (957/957), 8.96 MiB | 18.54 MiB/s, done.
Resolving deltas: 100% (457/457), done.
✓ Repository cloned
Current directory: /content/RFantibody


In [2]:
#@title 2. Download Model Weights
#@markdown Downloads the required model weights (~1.5 GB). This may take a few minutes.

#@title 2. Download Model Weights
# Download all weights
!bash include/download_weights.sh

# Copy to expected locations
!sudo mkdir -p /home/weights
!sudo cp /content/RFantibody/weights/* /home/weights/
!sudo cp /home/weights/RF2_ab.pt /home/weights/RFab_overall_best.pt

print("✓ Weights downloaded")
!ls /home/weights/

--2026-01-31 19:22:19--  https://files.ipd.uw.edu/pub/RFantibody/RFdiffusion_Ab.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.134, 128.95.160.135, 2607:4000:406::160:134, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 483452922 (461M) [application/octet-stream]
Saving to: ‘RFdiffusion_Ab.pt’

RFdiffusion_Ab.pt   100%[===================>] 461.06M  18.6MB/s    in 23s     

2026-01-31 19:22:44 (19.7 MB/s) - ‘RFdiffusion_Ab.pt’ saved [483452922/483452922]

--2026-01-31 19:22:44--  https://files.ipd.uw.edu/pub/RFantibody/ProteinMPNN_v48_noise_0.2.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.134, 128.95.160.135, 2607:4000:406::160:134, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6681301 (6.4M) [application/octet-stream]
Saving to: ‘ProteinMPNN_v48_noise_0.2.pt’

Pr

In [3]:
#@title 3. Setup Environment (Official UV Method)
#@markdown Installs dependencies using UV package manager. This may take several minutes.

import os

os.chdir("/content/RFantibody")

# Install uv package manager
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to PATH for this session
os.environ['PATH'] = f"/root/.local/bin:{os.environ['PATH']}"

# Sync environment (downloads Python 3.10 + all dependencies automatically)
!uv sync

# Clone and build USalign if missing
if not os.path.exists("/content/RFantibody/include/USalign"):
    !git clone https://github.com/pylelab/USalign.git /content/RFantibody/include/USalign

os.chdir("/content/RFantibody/include/USalign")
!make
os.chdir("/content/RFantibody")

# Verify installation
!uv run python -c "import torch; print(f'PyTorch: {torch.__version__}')"
!uv run python -c "import rfantibody; print('✓ RFantibody imported successfully')"

print("\n✓ Setup complete!")

downloading uv 0.9.28 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: .venv
Resolved 74 packages in 5.62s
Prepared 64 packages in 43.92s
Installed 65 packages in 213ms
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + asttokens==3.0.1
 + biotite==1.2.0
 + biotraj==1.2.2
 + certifi==2026.1.4
 + charset-normalizer==3.4.4
 + click==8.3.1
 + colorama==0.4.6
 + cuda-python==11.8.0
 + cython==3.2.4
 + dgl==2.4.0+cu118 (from https://data.dgl.ai/wheels/torch-2.3/cu118/dgl-2.4.0%2Bcu118-cp310-cp310-manylinux1_x86_64.whl)
 + e3nn==0.5.9
 + executing==2.2.1
 + filelock==3.20.3
 + fsspec==2026.1.0
 + hydra-core==1.3.2
 + icecream==2.1.10
 + idna==3.11
 + jinja2==3.1.6
 + markupsafe==3.0.3
 + mpmath==1.3.0
 + msgpack==1.1.2
 + networkx==3.4.2
 + numpy==1.26.4
 + nvidia-cublas-cu11==11.11.3.6
 + nvidia-cuda-cupti-cu11==11.8.87
 + nvidi

# RFantibody: Structure-Based *De Novo* Antibody Design

| Step | Model | Purpose |
|------|-------|--------|
| 1 | RFdiffusion (Ab) | Generate antibody-target docks |
| 2 | ProteinMPNN | Assign sequences to CDR loops |
| 3 | RF2 (Ab) | Predict structure and validate |

### Pipeline Flow
```
Target + Framework → RFdiffusion → ProteinMPNN → RF2
```

---
## Input Configuration

Upload your PDB files and set parameters below.

In [4]:
#@title 4. Create Input/Output Directories
#@markdown Creates folders to store your input files and results.

import os

os.makedirs("/content/inputs", exist_ok=True)
os.makedirs("/content/outputs", exist_ok=True)

In [8]:
#@title 5. Set Input Files
  #@markdown ### Upload Instructions:
  #@markdown 1. Click the folder icon on the left sidebar
  #@markdown 2. Navigate to `/content/inputs/`
  #@markdown 3. Right-click → Upload → select your file
  #@markdown ---

MODE = "nanobody" #@param ["nanobody", "antibody"]
TARGET_PDB = "/content/inputs/target_MAGEB1.pdb" #@param {type:"string"}
TARGET_CHAIN = "B" #@param {type:"string"}
FRAMEWORK_PDB = "/content/inputs/your_framework.pdb" #@param {type:"string"}

#@markdown > **Note:** For nanobody mode, the default framework is used automatically.

import os

if not os.path.exists(TARGET_PDB):
      raise FileNotFoundError(f"❌ File not found: {TARGET_PDB}\n   Upload your file first, then re-run this cell.")

with open(TARGET_PDB, 'r') as f:
      content = f.read()
content = content.replace(f' {TARGET_CHAIN} ', ' T ')
TARGET_PDB_RENAMED = "/content/inputs/target_T.pdb"
with open(TARGET_PDB_RENAMED, 'w') as f:
      f.write(content)

if MODE == "nanobody":
      FRAMEWORK_PDB = "/content/RFantibody/scripts/examples/example_inputs/h-NbBCII10.pdb"

print(f"✓ Mode: {MODE}")
print(f"✓ Chain {TARGET_CHAIN} renamed to T")
print(f"✓ Target: {TARGET_PDB_RENAMED}")
print(f"✓ Framework: {FRAMEWORK_PDB}")

✓ Mode: nanobody
✓ Chain B renamed to T
✓ Target: /content/inputs/target_T.pdb
✓ Framework: /content/RFantibody/scripts/examples/example_inputs/h-NbBCII10.pdb


In [10]:
#@title 6. Design Parameters
  #@markdown ---
  #@markdown ### Hotspot Residues (Epitope)
  #@markdown These are the residues on your target where you want the antibody to bind.
  #@markdown
  #@markdown **How to find them:**
  #@markdown 1. Open your target PDB in ChimeraX or PyMOL
  #@markdown 2. Identify the residues at your desired binding site
  #@markdown 3. Note the residue numbers (e.g., 195, 197, 256)
  #@markdown
  #@markdown **Format:** `[TResNum,TResNum,...]` — Use `T` prefix (we renamed your chain to T)
  #@markdown
  #@markdown **Example:** `[T195,T197,T256]`
HOTSPOT_RES = "[T195,T197,T254,T255,T256,T277,T279]" #@param {type:"string"}

  #@markdown ---
  #@markdown ### CDR Loops to Design
  #@markdown These are the antibody loops that will be designed to bind your target.
  #@markdown
  #@markdown | Mode | Loops | Description |
  #@markdown |------|-------|-------------|
  #@markdown | Nanobody | `[H1:7,H2:6,H3:5-13]` | 3 heavy chain loops only |
  #@markdown | Antibody | `[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]` | 6 loops |
  #@markdown
  #@markdown **Format:** `[LoopName:length]` or `[LoopName:min-max]` for variable length
DESIGN_LOOPS = "[H1:7,H2:6,H3:5-13]" #@param ["[H1:7,H2:6,H3:5-13]", "[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]"] {allow-input: true}

  #@markdown ---
  #@markdown ### Number of Designs
  #@markdown How many different antibody designs to generate.
  #@markdown
  #@markdown **Tip:** Start with 5-10 for testing, increase to 50+ for production runs.
NUM_DESIGNS = 5 #@param {type:"slider", min:1, max:50, step:1}

print(f"✓ Hotspot residues: {HOTSPOT_RES}")
print(f"✓ Design loops: {DESIGN_LOOPS}")
print(f"✓ Number of designs: {NUM_DESIGNS}")

✓ Hotspot residues: [T195,T197,T254,T255,T256,T277,T279]
✓ Design loops: [H1:7,H2:6,H3:5-13]
✓ Number of designs: 5


---
## Section 1: RFdiffusion

In [13]:
#@title 7. Run RFdiffusion
  #@markdown ---
  #@markdown ### What this does:
  #@markdown Generates antibody/nanobody backbones that dock to your target protein.
  #@markdown
  #@markdown **This step may take 5-15 minutes** depending on the number of designs.
  #@markdown ---

import os

os.chdir("/content/RFantibody")
os.makedirs("/content/outputs", exist_ok=True)

print("Running RFdiffusion...\n")

# Remove brackets from parameters for CLI
loops = DESIGN_LOOPS.strip('[]')
hotspots = HOTSPOT_RES.strip('[]')

!uv run rfdiffusion -t {TARGET_PDB_RENAMED} -f {FRAMEWORK_PDB} -o /content/outputs/ab_des -n {NUM_DESIGNS} -l "{loops}" -h "{hotspots}"

print(f"\n✓ RFdiffusion Complete!")
print(f"Generated files:")
for f in sorted(os.listdir("/content/outputs")):
      if f.endswith('.pdb'):
          print(f"  📄 {f}")

Running RFdiffusion...

Running RFdiffusion with 5 designs...
Target: /content/inputs/target_T.pdb
Framework: /content/RFantibody/scripts/examples/example_inputs/h-NbBCII10.pdb
Design loops: H1:7,H2:6,H3:5-13
/content/RFantibody/src/rfantibody/rfdiffusion/util.py:224: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  Z = torch.cross(Xn,Yn)
DGL backend not selected or invalid.  Assuming PyTorch for now.
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
/content/RFantibody/.venv/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'antibody': Defaults list 

---
## Section 2: ProteinMPNN

In [14]:
 #@title 8. Run ProteinMPNN
  #@markdown ---
  #@markdown ### What this does:
  #@markdown Designs amino acid sequences for the generated backbones from RFdiffusion.
  #@markdown
  #@markdown This converts the 3D backbone structures into actual protein sequences
  #@markdown that can be synthesized in the lab.
  #@markdown
  #@markdown ---

import os

os.chdir("/content/RFantibody")
os.makedirs("/content/outputs/mpnn", exist_ok=True)

print("Running ProteinMPNN...\n")

!uv run proteinmpnn -i /content/outputs -o /content/outputs/mpnn -n 1

print(f"\n✓ ProteinMPNN Complete!")
print(f"Generated files:")
for f in sorted(os.listdir("/content/outputs/mpnn")):
      if f.endswith('.pdb'):
          print(f"  📄 {f}")

# Show sequences
!uv run pip install -q biopython
from Bio.PDB import PDBParser

parser = PDBParser(QUIET=True)
print("\n=== Designed Sequences ===\n")
for f in sorted(os.listdir("/content/outputs/mpnn")):
      if f.endswith('.pdb'):
          print(f"--- {f} ---")
          structure = parser.get_structure("ab", f"/content/outputs/mpnn/{f}")
          for chain in structure[0]:
              seq = ''.join([res.resname for res in chain])
              print(f"  Chain {chain.id}: {len(list(chain))} residues")
          print()

Running ProteinMPNN...

Running ProteinMPNN sequence design...
Input: /content/outputs
Loops: H1,H2,H3,L1,L2,L3
Sequences per structure: 1
Found GPU will run ProteinMPNN on GPU
Attempting pose: /content/outputs/ab_des_1.pdb
loopH: [26, 27, 28, 29, 30, 31, 32, 52, 53, 54, 55, 56, 57, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108]
loopL: []
MPNN generated 1 sequences in 1 seconds
sequence_optimize: [('QVQLVESGGGLVQPGGSLRLSCAASGGDFSQYSLGWFRQAPGQGLEAVAAISADGTKTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAAKHSSDSGYKYWGQGTLVTVS', 1.3537776)]
Struct: /content/outputs/ab_des_1.pdb reported success in 1 seconds
Attempting pose: /content/outputs/ab_des_4.pdb
loopH: [26, 27, 28, 29, 30, 31, 32, 52, 53, 54, 55, 56, 57, 99, 100, 101, 102, 103]
loopL: []
MPNN generated 1 sequences in 1 seconds
sequence_optimize: [('QVQLVESGGGLVQPGGSLRLSCAASGADFSQYSLGWFRQAPGQGLEAVAAISADGTTTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAAGDLKYWGQGTLVTVS', 1.2249786)]
Struct: /content/outputs/ab_des_4.pdb reported success in 1 sec

---
## Section 3: RF2

In [16]:
#@title 9. RF2 Validation
  #@markdown ---
  #@markdown ### What this does:
  #@markdown Validates the designed structures using RoseTTAFold2.
  #@markdown
  #@markdown **Filtering criteria:**
  #@markdown - **pAE < 10** — Predicted Aligned Error (confidence score)
  #@markdown - **RMSD < 2Å** — Structural similarity to designed backbone
  #@markdown
  #@markdown Only designs that pass both filters will appear in the output.
  #@markdown If no files appear, try generating more designs in step 7.
  #@markdown
  #@markdown ---

import os

os.chdir("/content/RFantibody")
os.makedirs("/content/outputs/rf2", exist_ok=True)

print("Running RF2 Validation...\n")

!uv run rf2 -i /content/outputs/mpnn -o /content/outputs/rf2 -r 3

print(f"\n✓ RF2 Validation Complete!")
print(f"Generated files:")
files = [f for f in os.listdir("/content/outputs/rf2") if f.endswith('.pdb')]
if files:
      for f in sorted(files):
          print(f"  ✅ {f}")
      print(f"\n{len(files)} design(s) passed validation!")
else:
      print("  ⚠️ No files passed filtering (pAE < 10, RMSD < 2)")
      print("  💡 Try increasing NUM_DESIGNS and re-running from step 7")

Running RF2 Validation...

Running RF2 structure prediction...
Input: /content/outputs/mpnn
Recycles: 3
Hotspot show proportion: 0.1
Running RF2 with the following configs: {'input': {'pdb': None, 'pdb_dir': '/content/outputs/mpnn', 'quiver': None}, 'output': {'pdb_dir': '/content/outputs/rf2', 'quiver': None, 'output_intermediates': False}, 'inference': {'num_recycles': 3, 'hotspot_show_proportion': 0.1, 'cautious': True}, 'model': {'model_weights': '/content/RFantibody/weights/RF2_ab.pt'}, 'model_param': {'n_extra_block': 4, 'n_main_block': 36, 'd_msa': 256, 'd_pair': 128, 'd_templ': 64, 'n_head_msa': 8, 'n_head_pair': 4, 'n_head_templ': 4, 'd_hidden': 32, 'd_hidden_templ': 32, 'd_t1d': 23, 'p_drop': 0.0, 'SE3_param_full': {'num_layers': 1, 'num_channels': 48, 'num_degrees': 2, 'l0_in_features': 32, 'l0_out_features': 32, 'l1_in_features': 2, 'l1_out_features': 2, 'num_edge_features': 32, 'div': 4, 'n_heads': 4}, 'SE3_param_topk': {'num_layers': 1, 'num_channels': 128, 'num_degrees':



---


**Sequence Extraction with CDR Annotation**




In [19]:
#@title 10. Extract & Annotate Sequences
  #@markdown ---
  #@markdown ### What this does:
  #@markdown Extracts amino acid sequences from validated designs and annotates CDR regions.
  #@markdown
  #@markdown **Output includes:**
  #@markdown - Full VH/VHH sequences (and VL for antibodies)
  #@markdown - CDR1, CDR2, CDR3 regions (Chothia numbering)
  #@markdown - Framework regions (FR1-FR4)
  #@markdown - Excel file with all annotations
  #@markdown
  #@markdown ---
  #@markdown ### Configuration
  #@markdown > **Note:** This should match the MODE you selected in Step 5.

EXTRACTION_MODE = "nanobody" #@param ["nanobody", "antibody"]

  #@markdown ---

!pip install -q abnumber biopython pandas openpyxl

import os
import glob
import pandas as pd
from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1
from abnumber import Chain

RF2_OUTPUT_DIR = "/content/outputs/rf2"

def extract_sequence_from_pdb(pdb_file, chain_id):
      parser = PDBParser(QUIET=True)
      structure = parser.get_structure("protein", pdb_file)
      for model in structure:
          for chain in model:
              if chain.id == chain_id:
                  residues = [res for res in chain if res.id[0] == ' ']
                  sequence = ''.join([seq1(res.resname) for res in residues])
                  return sequence
      return None

def annotate_sequence(sequence, chain_type='H'):
      try:
          chain = Chain(sequence, scheme='chothia', chain_type=chain_type)
          cdr1_range = range(26, 33)
          cdr2_range = range(52, 57)
          cdr3_range = range(95, 103)
          fr1, cdr1, fr2, cdr2, fr3, cdr3, fr4 = [], [], [], [], [], [], []
          for pos, aa in chain:
              pos_num = pos.number
              if pos_num < 26:
                  fr1.append(aa)
              elif pos_num in cdr1_range:
                  cdr1.append(aa)
              elif pos_num < 52:
                  fr2.append(aa)
              elif pos_num in cdr2_range:
                  cdr2.append(aa)
              elif pos_num < 95:
                  fr3.append(aa)
              elif pos_num in cdr3_range:
                  cdr3.append(aa)
              else:
                  fr4.append(aa)
          regions = {
              'FR1': ''.join(fr1), 'CDR1': ''.join(cdr1),
              'FR2': ''.join(fr2), 'CDR2': ''.join(cdr2),
              'FR3': ''.join(fr3), 'CDR3': ''.join(cdr3),
              'FR4': ''.join(fr4)
          }
          return regions, True
      except Exception as e:
          print(f"    Warning: AbNumber failed - {e}")
          return None, False

def validate_length(sequence, chain_type='H'):
      length = len(sequence)
      if chain_type == 'H':
          return f"{length} residues ✓" if 100 <= length <= 140 else f"{length} residues ⚠ (expected 100-140)"
      else:
          return f"{length} residues ✓" if 100 <= length <= 130 else f"{length} residues ⚠ (expected 100-130)"

pdb_files = sorted(glob.glob(os.path.join(RF2_OUTPUT_DIR, "*.pdb")))

if not pdb_files:
      print(f"❌ No PDB files found in {RF2_OUTPUT_DIR}")
      print("   Make sure you've run RF2 validation first!")
else:
      print(f"Found {len(pdb_files)} RF2 output files\n")

all_sequences = []

for pdb_file in pdb_files:
      design_name = os.path.basename(pdb_file).replace('.pdb', '')
      print("═" * 60)
      print(f"DESIGN: {design_name}")
      print("═" * 60)
      design_data = {'Design': design_name}
      vh_seq = extract_sequence_from_pdb(pdb_file, 'H')
      if vh_seq:
          vh_valid = validate_length(vh_seq, 'H')
          print(f"\nHEAVY CHAIN (VH/VHH): {vh_valid}")
          print(f"\n>>{design_name}_VH")
          print(vh_seq)
          vh_regions, success = annotate_sequence(vh_seq, 'H')
          if success:
              print(f"\nREGION BREAKDOWN (Chothia):")
              print(f"├─ FR1:  {vh_regions['FR1']}")
              print(f"├─ CDR1: {vh_regions['CDR1']}")
              print(f"├─ FR2:  {vh_regions['FR2']}")
              print(f"├─ CDR2: {vh_regions['CDR2']}")
              print(f"├─ FR3:  {vh_regions['FR3']}")
              print(f"├─ CDR3: {vh_regions['CDR3']}")
              print(f"└─ FR4:  {vh_regions['FR4']}")
              print(f"\nCDRs ONLY (for analysis tools):")
              print(f"  CDR-H1: {vh_regions['CDR1']}")
              print(f"  CDR-H2: {vh_regions['CDR2']}")
              print(f"  CDR-H3: {vh_regions['CDR3']}")
              design_data['VH_Full'] = vh_seq
              design_data['VH_Length'] = len(vh_seq)
              design_data['VH_FR1'] = vh_regions['FR1']
              design_data['VH_CDR1'] = vh_regions['CDR1']
              design_data['VH_FR2'] = vh_regions['FR2']
              design_data['VH_CDR2'] = vh_regions['CDR2']
              design_data['VH_FR3'] = vh_regions['FR3']
              design_data['VH_CDR3'] = vh_regions['CDR3']
              design_data['VH_FR4'] = vh_regions['FR4']
          else:
              design_data['VH_Full'] = vh_seq
              design_data['VH_Length'] = len(vh_seq)
      if EXTRACTION_MODE == "antibody":
          vl_seq = extract_sequence_from_pdb(pdb_file, 'L')
          if vl_seq:
              vl_valid = validate_length(vl_seq, 'L')
              print(f"\nLIGHT CHAIN (VL): {vl_valid}")
              print(f"\n>>{design_name}_VL")
              print(vl_seq)
              vl_regions, success = annotate_sequence(vl_seq, 'L')
              if success:
                  print(f"\nREGION BREAKDOWN (Chothia):")
                  print(f"├─ FR1:  {vl_regions['FR1']}")
                  print(f"├─ CDR1: {vl_regions['CDR1']}")
                  print(f"├─ FR2:  {vl_regions['FR2']}")
                  print(f"├─ CDR2: {vl_regions['CDR2']}")
                  print(f"├─ FR3:  {vl_regions['FR3']}")
                  print(f"├─ CDR3: {vl_regions['CDR3']}")
                  print(f"└─ FR4:  {vl_regions['FR4']}")
                  print(f"\nCDRs ONLY:")
                  print(f"  CDR-L1: {vl_regions['CDR1']}")
                  print(f"  CDR-L2: {vl_regions['CDR2']}")
                  print(f"  CDR-L3: {vl_regions['CDR3']}")
                  design_data['VL_Full'] = vl_seq
                  design_data['VL_Length'] = len(vl_seq)
                  design_data['VL_FR1'] = vl_regions['FR1']
                  design_data['VL_CDR1'] = vl_regions['CDR1']
                  design_data['VL_FR2'] = vl_regions['FR2']
                  design_data['VL_CDR2'] = vl_regions['CDR2']
                  design_data['VL_FR3'] = vl_regions['FR3']
                  design_data['VL_CDR3'] = vl_regions['CDR3']
                  design_data['VL_FR4'] = vl_regions['FR4']
              else:
                  design_data['VL_Full'] = vl_seq
                  design_data['VL_Length'] = len(vl_seq)
      all_sequences.append(design_data)
      print()

if all_sequences:
      sequences_df = pd.DataFrame(all_sequences)
      output_file = "/content/outputs/sequences_annotated.xlsx"
      sequences_df.to_excel(output_file, index=False)
      print("═" * 60)
      print("SUMMARY")
      print("═" * 60)
      print(f"✓ Extracted {len(all_sequences)} designs")
      print(f"✓ Saved to: {output_file}")
      print(f"\nColumns in Excel:")
      for col in sequences_df.columns:
          print(f"  • {col}")
      print(f"\n📋 Variable 'sequences_df' ready for analysis cells")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 113.1 MB/s eta 0:00:00
Found 5 RF2 output files

════════════════════════════════════════════════════════════
DESIGN: ab_des_0_dldesign_0_best
════════════════════════════════════════════════════════════

HEAVY CHAIN (VH/VHH): 118 residues ✓

>>ab_des_0_dldesign_0_best_VH
QVQLVESGGGLVQPGGSLRLSCAASGGDFSQYSLGWFRQAPGQGLEAVAAISSDGTTTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAAGPSLDLGPVYWGQGTLVTVS

════════════════════════════════════════════════════════════
DESIGN: ab_des_1_dldesign_0_best
════════════════════════════════════════════════════════════

HEAVY CHAIN (VH/VHH): 118 residues ✓

>>ab_des_1_dldesign_0_best_VH
QVQLVESGGGLVQPGGSLRLSCAASGGDFSQYSLGWFRQAPGQGLEAVAAISADGTKTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAAKHSSDSGYKYWGQGTLVTVS

════════════════════════════════════════════════════════════
DESIGN: ab_des_2_dldesign_0_best
════════════════════════════════════

---
## Section 4: Results

In [21]:
#@title 11. Download Results
  #@markdown ---
  #@markdown ### Choose what to download:
  #@markdown | Option | Contents |
  #@markdown |--------|----------|
  #@markdown | RF2 | Final validated antibodies |
  #@markdown | RF2+MPNN | Validated + sequence designs |
  #@markdown | All | RFdiffusion + ProteinMPNN + RF2 |
  #@markdown ---

DOWNLOAD_OPTION = "RF2" #@param ["RF2", "RF2+MPNN", "All"]

import shutil
import os
from google.colab import files

os.makedirs("/content/download_temp", exist_ok=True)
count = 0

rf2_count = 0
for f in os.listdir("/content/outputs/rf2"):
      if f.endswith('.pdb'):
          shutil.copy(f"/content/outputs/rf2/{f}",
  "/content/download_temp/")
          rf2_count += 1
print(f"✓ Added {rf2_count} RF2 files")
count += rf2_count

if DOWNLOAD_OPTION in ["RF2+MPNN", "All"]:
      mpnn_count = 0
      for f in os.listdir("/content/outputs/mpnn"):
          if f.endswith('.pdb'):
              shutil.copy(f"/content/outputs/mpnn/{f}",
  "/content/download_temp/")
              mpnn_count += 1
      print(f"✓ Added {mpnn_count} ProteinMPNN files")
      count += mpnn_count

if DOWNLOAD_OPTION == "All":
      rfd_count = 0
      for f in os.listdir("/content/outputs"):
          if f.endswith('.pdb'):
              shutil.copy(f"/content/outputs/{f}",
  "/content/download_temp/")
              rfd_count += 1
      print(f"✓ Added {rfd_count} RFdiffusion files")
      count += rfd_count

if os.path.exists("/content/outputs/sequences_annotated.xlsx"):
      shutil.copy("/content/outputs/sequences_annotated.xlsx",
  "/content/download_temp/")
      print("✓ Added sequences_annotated.xlsx")

shutil.make_archive('/content/rfantibody_results', 'zip',
  '/content/download_temp')
files.download('/content/rfantibody_results.zip')

print(f"\n📦 Total: {count} PDB files downloaded")
shutil.rmtree("/content/download_temp")

✓ Added 5 RF2 files
✓ Added sequences_annotated.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


📦 Total: 5 PDB files downloaded
